In [137]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np




In [138]:
## Load the trained model, scaler, pickle, onehot

model=load_model('model.keras')

## load encders and scalers

with open('onehot_encoder_geo.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)
    
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)
    
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)
    

In [139]:
#Example input data


input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

#How do you apply the Categorical Encoding and Scaling to the input data?

# onehot encode 'Geography'

#temporary generate a data_frame from the input data
temp_df=pd.DataFrame([input_data])

# So it is a Data Frame
geo_encoded=label_encoder_geo.transform(temp_df[['Geography']])

geo_encoded_df=pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))

#We need to convert to a data frame and

input_df=pd.DataFrame([input_data])


In [140]:
#Encode categorical variables

input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])


In [ ]:
# Verify that the change in gender was don correctly

input_df

In [142]:
# Concatenation of Onhot encoded and input data 

# First drop Geography 
input_df=pd.concat([input_df.drop("Geography",axis=1)])


In [ ]:
#View input_df
input_df

In [144]:
# Now add Geography Encoding

input_df=pd.concat([input_df,geo_encoded_df],axis=1)    

In [ ]:
# View input_df
input_df

In [132]:
# Scaling the input data to the same scale as the training data

input_scaled=scaler.transform(input_df)

In [ ]:
# View the input_scaled

input_scaled

In [148]:
# Define the prediction function with reduce_retracing=True
@tf.function(reduce_retracing=True)
def predict(input_tensor):
    return model(input_tensor)

# Predict churn

prediction=model.predict(input_scaled)

prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


array([[0.03062999]], dtype=float32)

In [149]:
# Get prediction probability

prediction_prob=prediction[0][0]

prediction_prob

0.030629989

In [150]:
# Results

if prediction_prob>0.5:
    print("Customer is likely to churn")
else:
    print("Customer is not likely to churn")

Customer is not likely to churn
